# Downsampling Intan dat files

Creates a copy of each channel file undersampled by 10.

Method:
IRR filter

Creates a subfolder  'downsampleLFP'

Filenames: '_sr_2500.dat'




In [1]:
from tqdm.notebook import tqdm
import numpy as np
from scipy import signal
import sys, struct, os
from glob import glob
import mmap
import gc
gc.enable()

In [27]:
def read_qstring(fid):
    """Read Qt style QString.  

    The first 32-bit unsigned number indicates the length of the string (in bytes).  
    If this number equals 0xFFFFFFFF, the string is null.

    Strings are stored as unicode.
    """
    length, = struct.unpack('<I', fid.read(4))
    if length == int('ffffffff', 16): return ""

    if length > (os.fstat(fid.fileno()).st_size - fid.tell() + 1) :
        print(length)
        raise Exception('Length too long.')
    # convert length from bytes to 16-bit Unicode words
    length = int(length / 2)
    data = []
    for i in range(0, length):
        c, = struct.unpack('<H', fid.read(2))
        data.append(c)
    if sys.version_info >= (3,0):
        a = ''.join([chr(c) for c in data])
    else:
        a = ''.join([unichr(c) for c in data])
    return a

def read_header(fid):
    """Reads the Intan File Format header from the given file."""

    # Check 'magic number' at beginning of file to make sure this is an Intan
    # Technologies RHD2000 data file.
    magic_number, = struct.unpack('<I', fid.read(4)) 
    if magic_number != int('c6912702', 16): raise Exception('Unrecognized file type.')

    header = {}
    # Read version number.
    version = {}
    (version['major'], version['minor']) = struct.unpack('<hh', fid.read(4)) 
    header['version'] = version

    # print('')
    # print('Reading Intan Technologies RHD2000 Data File, Version {}.{}'.format(version['major'], version['minor']))
    # print('')

    freq = {}

    # Read information of sampling rate and amplifier frequency settings.
    header['sample_rate'], = struct.unpack('<f', fid.read(4))
    (freq['dsp_enabled'], freq['actual_dsp_cutoff_frequency'], freq['actual_lower_bandwidth'], freq['actual_upper_bandwidth'], 
    freq['desired_dsp_cutoff_frequency'], freq['desired_lower_bandwidth'], freq['desired_upper_bandwidth']) = struct.unpack('<hffffff', fid.read(26))


    # This tells us if a software 50/60 Hz notch filter was enabled during
    # the data acquisition.
    notch_filter_mode, = struct.unpack('<h', fid.read(2))
    header['notch_filter_frequency'] = 0
    if notch_filter_mode == 1:
        header['notch_filter_frequency'] = 50
    elif notch_filter_mode == 2:
        header['notch_filter_frequency'] = 60
    freq['notch_filter_frequency'] = header['notch_filter_frequency']

    (freq['desired_impedance_test_frequency'], freq['actual_impedance_test_frequency']) = struct.unpack('<ff', fid.read(8))

    note1 = read_qstring(fid)
    note2 = read_qstring(fid)
    note3 = read_qstring(fid)
    header['notes'] = { 'note1' : note1, 'note2' : note2, 'note3' : note3}

    # If data file is from GUI v1.1 or later, see if temperature sensor data was saved.
    header['num_temp_sensor_channels'] = 0
    if (version['major'] == 1 and version['minor'] >= 1) or (version['major'] > 1) :
        header['num_temp_sensor_channels'], = struct.unpack('<h', fid.read(2))
        
    # If data file is from GUI v1.3 or later, load eval board mode.
    header['eval_board_mode'] = 0
    if ((version['major'] == 1) and (version['minor'] >= 3)) or (version['major'] > 1) :
        header['eval_board_mode'], = struct.unpack('<h', fid.read(2))
        
        
    header['num_samples_per_data_block'] = 60
    # If data file is from v2.0 or later (Intan Recording Controller), load name of digital reference channel
    if (version['major'] > 1):
        header['reference_channel'] = read_qstring(fid)
        header['num_samples_per_data_block'] = 128

    # Place frequency-related information in data structure. (Note: much of this structure is set above)
    freq['amplifier_sample_rate'] = header['sample_rate']
    freq['aux_input_sample_rate'] = header['sample_rate'] / 4
    freq['supply_voltage_sample_rate'] = header['sample_rate'] / header['num_samples_per_data_block']
    freq['board_adc_sample_rate'] = header['sample_rate']
    freq['board_dig_in_sample_rate'] = header['sample_rate']

    header['frequency_parameters'] = freq

    # Create structure arrays for each type of data channel.
    header['spike_triggers'] = []
    header['amplifier_channels'] = []
    header['aux_input_channels'] = []
    header['supply_voltage_channels'] = []
    header['board_adc_channels'] = []
    header['board_dig_in_channels'] = []
    header['board_dig_out_channels'] = []

    # Read signal summary from data file header.

    number_of_signal_groups, = struct.unpack('<h', fid.read(2))
    # print('n signal groups {}'.format(number_of_signal_groups))

    for signal_group in range(1, number_of_signal_groups + 1):
        signal_group_name = read_qstring(fid)
        signal_group_prefix = read_qstring(fid)
        (signal_group_enabled, signal_group_num_channels, signal_group_num_amp_channels) = struct.unpack('<hhh', fid.read(6))

        if (signal_group_num_channels > 0) and (signal_group_enabled > 0):
            for signal_channel in range(0, signal_group_num_channels):
                new_channel = {'port_name' : signal_group_name, 'port_prefix' : signal_group_prefix, 'port_number' : signal_group}
                new_channel['native_channel_name'] = read_qstring(fid)
                new_channel['custom_channel_name'] = read_qstring(fid)
                (new_channel['native_order'], new_channel['custom_order'], signal_type, channel_enabled, new_channel['chip_channel'], new_channel['board_stream']) = struct.unpack('<hhhhhh', fid.read(12))
                new_trigger_channel = {}
                (new_trigger_channel['voltage_trigger_mode'], new_trigger_channel['voltage_threshold'], new_trigger_channel['digital_trigger_channel'], new_trigger_channel['digital_edge_polarity'])  = struct.unpack('<hhhh', fid.read(8))
                (new_channel['electrode_impedance_magnitude'], new_channel['electrode_impedance_phase']) = struct.unpack('<ff', fid.read(8))

                if channel_enabled:
                    if signal_type == 0:
                        header['amplifier_channels'].append(new_channel)
                        header['spike_triggers'].append(new_trigger_channel)
                    elif signal_type == 1:
                        header['aux_input_channels'].append(new_channel)
                    elif signal_type == 2:
                        header['supply_voltage_channels'].append(new_channel)
                    elif signal_type == 3:
                        header['board_adc_channels'].append(new_channel)
                    elif signal_type == 4:
                        header['board_dig_in_channels'].append(new_channel)
                    elif signal_type == 5:
                        header['board_dig_out_channels'].append(new_channel)
                    else:
                        raise Exception('Unknown channel type.')
                        
    # Summarize contents of data file.
    header['num_amplifier_channels'] = len(header['amplifier_channels'])
    header['num_aux_input_channels'] = len(header['aux_input_channels'])
    header['num_supply_voltage_channels'] = len(header['supply_voltage_channels'])
    header['num_board_adc_channels'] = len(header['board_adc_channels'])
    header['num_board_dig_in_channels'] = len(header['board_dig_in_channels'])
    header['num_board_dig_out_channels'] = len(header['board_dig_out_channels'])

    return header

In [35]:
# # Check if all sample rates are the same
# for head in glob("/mnt/g/HUMAN/**/*.rhd", recursive = True):
#     h=read_header(open('/mnt/g/HUMAN/HUMAN 212/slice1_151002_212538/info.rhd', 'rb'))
#     sr = h['sample_rate']
#     print(f"{head.split('/')[-3:-1]} : {int(sr)}")
#     if sr != 25000.:
#         print('OPAIH!')

In [36]:
humans_to_downsample =[
                '/mnt/g/HUMAN/HUMAN 212/*/',
                '/mnt/g/HUMAN/HUMAN 213/',
                '/mnt/g/HUMAN/HUMAN 214/',
                '/mnt/g/HUMAN/HUMAN 215/',
                '/mnt/g/HUMAN/HUMAN 216/',
                '/mnt/g/HUMAN/HUMAN 217/',
                '/mnt/g/HUMAN/HUMAN 218/',
                '/mnt/g/HUMAN/HUMAN 219/',
                '/mnt/g/HUMAN/HUMAN 220/',
                '/mnt/g/HUMAN/HUMAN 226/',
                '/mnt/g/HUMAN/HUMAN 227/',
                '/mnt/g/HUMAN/HUMAN 229/',
                '/mnt/g/HUMAN/HUMAN 230/',
                '/mnt/g/HUMAN/HUMAN 231/',
                '/mnt/g/HUMAN/HUMAN 238/',
                '/mnt/g/HUMAN/HUMAN 241/'
            ]

In [38]:
# Config
new_path = 'downsampleLFP/'

In [6]:
def downsample(path):
    head_file = glob(path+"/info.rhd/")
    for data_path in tqdm(glob(path+"/*/", recursive = True)):
        if not os.path.exists(data_path + new_path):
                os.makedirs(data_path + new_path)
        print(f'Resampling channels of {data_path.split("/")[-3]} slide {data_path.split("/")[-2]}\n\n')
        for filename in tqdm(sorted(glob(data_path+'amp*.dat'))):
            new_name = data_path + new_path + os.path.basename(filename).split('.')[-2]+'_sr_1000.dat'
            b = np.memmap(filename, dtype='int16')*.195
            if os.path.isfile(new_name) and os.path.getsize(new_name) == \
                                os.path.getsize(filename)/10:
                continue
            else:
                downSample = signal.decimate(b, 25,ftype='fir')
                fp = np.memmap(new_name, dtype='int16', 
                                mode='w+', offset=0, shape=downSample.shape)
                fp[:] = downSample[:]
                fp.flush()
    print('Done!')

  0%|          | 0/3 [00:00<?, ?it/s]

Resampling channels of HUMAN 230 slide slide_1_2




  0%|          | 0/128 [00:00<?, ?it/s]

Resampling channels of HUMAN 230 slide slide_3_4




  0%|          | 0/128 [00:00<?, ?it/s]

Resampling channels of HUMAN 230 slide slide_5_6




  0%|          | 0/128 [00:00<?, ?it/s]